In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
## Change these variables to your desired values
img_height = 256
img_width = 256
img_channels = 1
    
path_labels = "/Users/megantabbutt/Desktop/Computer Science Classes/762_AdvancedDeepLearning/\
762_Project/Codes/Paper_codes/labels/"  # Where the labels are saved   
path_videos = "/Users/megantabbutt/Desktop/Computer Science Classes/762_AdvancedDeepLearning/\
762_Project/Codes/Paper_codes/video data_save" # Where the videos are saved 
model_stable_path = './models/model.h5' # From where to load the CNN before training
model_save_path = './models/model.h5' # Where to save the CNN after training

#n=100000 # if you want to limit to a small subset of the data
#n=1000
n=300
test_size=0.1
val_size=0.1
batch_size = 16    
epochs=2 # For how many epochs to train


FLAG_DEBUG = False
FLAG_GENERATE_TABLE=True # If false: load a previously generated table
FLAG_TRAIN=True # If false: just load a model, do not retrain
FLAG_TRAIN_IN_PRELOAD=False #if false: train with lazy loading (see details in Training Modes section)

label_names=['Human',
             'Interaction frontal',
             'Interaction lateral', 
             'Interaction vertical',
             'Crowded', 
             'Drink',
             'Curiosity', 
             'Queue',
             'Low visibility', 
             'Nothing']

In [ ]:
# Math manipulation
import numpy as np
import pandas as pd

# Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Image processing
import cv2

# Utilities
import os
import random
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical
import codecs, json 
import re

In [ ]:
def generate_pointer_table (list_of_videos, path_labels, plot=True, verbose=False):
    pt_table = []
    for filename in list_of_videos:
        with open(filename) as json_file: file_labels = json.load(json_file)[1:]
        i=1
        name = re.search(str(path_labels + '(.+?).json'), filename).group(1)
        if 'pointer_table' in name: continue
        base_time = int(name) 
        while i<len(file_labels):
            if file_labels[i]!=-1: pt_table.append([base_time, i, file_labels[i]])
            i +=1
    return pt_table

In [ ]:
#def preprocess_frame(frame):
#    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # convert to greyscale
#    frame = cv2.resize (frame, (img_width, img_height), interpolation=cv2.INTER_CUBIC) # rezize
#    return frame

def preprocess_frame(frame):
    if type(frame) != int and len(frame.shape)>1:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # convert to greyscale
    frame = cv2.resize (frame, (img_width, img_height), interpolation=cv2.INTER_CUBIC) # rezize
    return frame

In [ ]:
def frame_from_pointer (path, video, frame, verbose=False):
    if verbose: print ('In '+path+'/'+str(video)+'.mp4' + ', taking frame ' + str(frame))
    cap = cv2.VideoCapture(path+'/'+str(video)+'.mp4')
    if (cap.isOpened()== False):
        print("Error opening video file") 
        return -1
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    ret, frame = cap.read() # Capture next frame
    if ret==True:
        cap.release()
        return frame
    else:
        print("Error opening frame") 
        return -1

In [ ]:
# need to delete the videos from the labels that they removed:

for dirs in os.listdir(path_labels):
    if dirs[:-5]+".mp4" not in os.listdir(path_videos):
        print(dirs)
        if len(dirs) == 15:
            os.remove(path_labels+dirs)
        

In [ ]:
list_of_videos = [path_labels + i for i in sorted(os.listdir(path_labels))]
pointer_table_path = path_labels + 'pointer_table.json'
pointer_table_shuffled_path = path_labels + 'pointer_table_shuffled.json'
if FLAG_DEBUG: print ('Number of files: ', len(list_of_videos))
    
if FLAG_GENERATE_TABLE:   
    pt_table = generate_pointer_table (list_of_videos, path_labels)
    json.dump(pt_table, codecs.open(pointer_table_path, 'w', encoding='utf-8'))
    print ('Pointer table saved')
    random.shuffle(pt_table)
    json.dump(pt_table, codecs.open(pointer_table_shuffled_path, 'w', encoding='utf-8'))
    print ('Shuffled pointer table saved')
    
else:
    with open(pointer_table_shuffled_path) as json_file: pt_table = json.load(json_file)
    #random.shuffle(pt_table)  #if you want to shuffle again
    
pt_table = np.array(pt_table)


train_max_index = int(n*(1-test_size - val_size))
val_max_index = int(train_max_index + n*val_size)
test_max_index = int(val_max_index + n*test_size)

pt_table_train = pt_table[0:train_max_index]
pt_table_val = pt_table[train_max_index:val_max_index]
pt_table_test = pt_table[val_max_index:test_max_index]

if FLAG_DEBUG:
    print ('Train index = ',0,':', train_max_index)
    print ('Val index   = ', train_max_index,':', val_max_index)
    print ('Test index  = ', val_max_index,':', test_max_index)


#del list_of_videos, pt_table
gc.collect()

# Training modes
To improve the training process, we developed two different modes of training that differ in the way the data is handled during the training: pre-loading the images or loading just-in-time. 

The **pre-loaded mode** loads the images into the computer's temporary memory only at the beginning of the training phase. This allows faster training; nevertheless, it was not possible to load more than 40k images at the same time without available hardware.

The **just-in-time loading** (also designated in the documentation as lazy loading) retrieves the images from disk every time they are required in the training phase. This allows training with the entire dataset, no matter how big it is, but has the drawback that the same image had to be loaded several times (for instance: to train 10 epochs is necessary to load the same image 10 times).

To change the training mode is enough to change the varible FLAG_TRAIN_IN_PRELOAD value. To train the network presented in this work, we used the just-in-time loading mode.

In [ ]:
from albumentations import Compose
import albumentations as A
import random

augmentation_train = A.Compose([
    A.RandomContrast(limit=0.3, p=0.5),
    A.RandomGamma(gamma_limit=(70, 130), p=0.5),
    A.RandomBrightness(limit=0.6, p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.05, 
        scale_limit=0.05, 
        rotate_limit=10, 
        border_mode=cv2.BORDER_CONSTANT, 
        p=0.5), 
    A.ToFloat(max_value=255),
])

augmentation_test = A.Compose([
    A.ToFloat(max_value=255)
])

In [ ]:
## Preload import 
# require more RAM, but train fast
# verbose:   0=quiet, 1=inline output, 2=save figs
def preload_import (pt_table, X_path, verbose=1):
    X = []
    y = []
    for pointer in pt_table:
        #print ('.')
        frame = frame_from_pointer(path=X_path, video=pointer[0], frame=pointer[1])
        frame = preprocess_frame(frame)
        X.append(frame)
        y.append(to_categorical(pointer[2], num_classes=10))

    X = np.array(X)
    y = np.array(y)  
    if verbose>=1:
        print('------------------------')
        print('Some random pics:')
        plt.figure(figsize=(100,100))
        cols = 4
        rows = 4
        for c in range(cols):
            for r in range(rows):
                plt.subplot(rows, cols, c + cols*r + 1)
                plt.imshow(X[c + cols*r])
                plt.title(label_names[np.argmax(y[c + cols*r])], fontsize=102)
                
                plt.axis('off')
        if verbose>1: plt.savefig('sample_randoms.png')
            
    return X,y

In [ ]:
if FLAG_TRAIN_IN_PRELOAD:
    X_train, y_train = preload_import (pt_table_train, X_path=path_videos, verbose=1)
    print('Train dataset loaded')
    X_val, y_val = preload_import (pt_table_val, X_path=path_videos, verbose=1)
    print('Val dataset loaded')    
    print('-- Split report (np shapes) --' + 
         '\nTrain X      : ' + str(X_train.shape) + 
         '\nTrain y      : ' + str(y_train.shape) + 
         '\nValidation X : ' + str(X_val.shape) + 
         '\nValidation y : ' + str(y_val.shape))

In [ ]:
def image_generator_preloaded (X_train, y_train, batch_size, augmentation): 
    indices = np.arange(len(X_train)) 
    while True:
        batch_indices = np.random.choice(indices, batch_size, replace=False)
        batch_X = []
        batch_y = []
        
        for i in batch_indices:
            frame = X_train[i]
            frame = augmentation(image=frame)['image']
            frame = frame.reshape(frame.shape[0], frame.shape[1], img_channels)
            batch_X.append(frame)
            batch_y.append(y_train[i])
            
        # Return a tuple of (input,output) to feed the network   
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        yield(batch_X, batch_y)

if FLAG_TRAIN_IN_PRELOAD:
    #get the length of the train and validation data
    ntrain = len(X_train)
    nval = len(X_val)

    train_generator = image_generator_preloaded (X_train, y_train, batch_size=batch_size, augmentation=augmentation_train)
    val_generator = image_generator_preloaded (X_train, y_train, batch_size=batch_size, augmentation=augmentation_test)

In [ ]:
def image_generator_lazy (pt_table, batch_size, augmentation):
    while True:          # Select files (paths/indices) for the batch
        pointer_batch = pt_table[np.random.choice(pt_table.shape[0], batch_size, replace=False), :]
        batch_X = []
        batch_y = []
        
        for pointer in pointer_batch:
            frame = frame_from_pointer(path=path_videos, video=pointer[0], frame=pointer[1])
            #print(type(frame))
            #if type(frame) != int: print(frame.shape)
            frame = preprocess_frame(frame)
            frame = frame.astype(np.float32) # MT added 
            frame = augmentation(image=frame)['image']
            frame = frame.reshape(frame.shape[0], frame.shape[1], img_channels)
            label = to_categorical(pointer[2], num_classes=10)
        
            batch_X.append(frame)
            batch_y.append(label)
             
        # Return a tuple of (input,output) to feed the network          
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        yield(batch_X, batch_y)

if not FLAG_TRAIN_IN_PRELOAD:
    #get the length of the train and validation data
    ntrain = len(pt_table_train)
    nval = len(pt_table_val)

    train_generator = image_generator_lazy (pt_table_train, batch_size=batch_size, augmentation=augmentation_train)
    val_generator = image_generator_lazy (pt_table_val, batch_size=batch_size, augmentation=augmentation_test)

In [ ]:
# Visualizing one batch
if FLAG_DEBUG:
    batch_X, batch_y = next(train_generator)
    print(batch_X.shape)
    print(batch_y.shape)

    plt.figure(figsize=(100,100))
    cols = 4
    rows = 4
    for c in range(cols):
        for r in range(rows):
            plt.subplot(rows, cols, c + cols*r + 1)
            img = batch_X[c + cols*r].reshape(batch_X[c + cols*r].shape[0], batch_X[c + cols*r].shape[1])
            print ('Frame stats: ', img.mean(), ' - ', img.std())
            plt.imshow(img)
            plt.title(label_names[np.argmax(batch_y[c + cols*r])], fontsize=102)
            plt.axis('off')

# Model definition and training
* That architecture is based on VGGnet architecture, with a few modifications (like dropout)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

#utils
from keras_tqdm import TQDMNotebookCallback

# Models
from keras import applications
from keras.optimizers import SGD, Adam

In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, img_channels)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),


    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),


    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy']) 
    
model.summary()

In [ ]:
#model = load_model(model_stable_path) # load last stable model (can be commented to train from scrach)
initial_epoch=0 # MT added
    
if FLAG_TRAIN: 
    callback_es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=20)
    callback_cp = ModelCheckpoint(model_save_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    history = model.fit_generator(train_generator,
                                  steps_per_epoch=ntrain // batch_size,
                                  epochs=initial_epoch+epochs,
                                  initial_epoch=initial_epoch,
                                  validation_data=val_generator,
                                  validation_steps=nval // batch_size,
                                  verbose=0, 
                                  callbacks=[TQDMNotebookCallback(), callback_es, callback_cp])

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation acc
plt.grid(True)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
plt.figure()

#Train and validation loss
plt.grid(True)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

# Test

In [ ]:
if FLAG_TRAIN:
    model = load_model(model_save_path) # load the best model saved in training phase

In [ ]:
# The test should be done with pre loading, so I will take directly from the generator 
test_generator = image_generator_lazy (pt_table_test, batch_size=batch_size, augmentation=augmentation_test)

X_test = []
y_test = []
predictions = []
for i in range(0, int(len(pt_table_test)/batch_size)):
    X_batch, y_batch = next(test_generator)
    predictions_batch = model.predict_on_batch(X_batch)
    X_test.extend(X_batch)
    y_test.extend(y_batch)
    predictions.extend(predictions_batch)

X_test = np.array (X_test)
y_test = np.array (y_test)
predictions = np.array (predictions)

In [ ]:
_, acc = model.evaluate(X_test, y_test)
print('Test acc:', acc)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# Brief: evaluate the 'accuracy' as considering a hit if the true label is in one of the three most confident predictions
# Return: acc
def evaluate_minority_acc (y, y_hat):
    acc = 0
    for i in range(0, len(y), 1):
        pred_array = []
        pred_array.append(np.where(y_hat[i] == sorted(y_hat[i])[-1])[0][0])
        pred_array.append(np.where(y_hat[i] == sorted(y_hat[i])[-2])[0][0])
        pred_array.append(np.where(y_hat[i] == sorted(y_hat[i])[-3])[0][0])
        if np.argmax(y[i]) in pred_array: acc += 1
    return acc/len (y)
    
    
evaluate_minority_acc(y_test, predictions)

In [ ]:
# Brief: plot the image with the predicted label, confidence and real label
def plot_image(i, prediction, true_label, img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img.reshape(img.shape[0], img.shape[1]))

    if np.argmax(prediction) == np.argmax(true_label):
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{}) {}, {:2.0f}% \n({})".format(
        i,
        label_names[np.argmax(prediction)],
        100*np.max(prediction),
        label_names[np.argmax(true_label)]), color=color)

# Brief: plot a bar graph with the probability of each predicted class
def plot_value_array(prediction, true_label):
    plt.grid(False)
    plt.xticks(range(10))
    thisplot = plt.bar(range(10), prediction, color="#777777")

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
if FLAG_DEBUG:
    num_rows = 160
    num_cols = 1
    num_images = num_rows*num_cols
    plt.figure(figsize=(7, 4*num_rows))
    for i in range(num_images):
        plt.subplot(num_rows, 2*num_cols, 2*i+1)
        plot_image(i, predictions[i], y_test[i], X_test[i])
        plt.subplot(num_rows, 2*num_cols, 2*i+2)
        plot_value_array(predictions[i],  y_test[i])
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred, figsize=(10,10)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax, linewidths=.5)

plot_cm(np.array([np.argmax(xi) for xi in y_test]), np.array([np.argmax(xi) for xi in predictions]))
_n=0
for i in label_names:
    print('Class ' + str(_n) + ': ' + i)
    _n+=1

In [ ]:
import sklearn.metrics as metrics
print(metrics.classification_report(np.array([np.argmax(xi) for xi in y_test]), np.array([np.argmax(xi) for xi in predictions]), target_names=label_names))

License: Creative Commons 4.0 Attribute